In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [104]:
df = pd.read_csv('Trading_Project_Data.csv') 
df.head() 


,Price,Close,Close.1,Close.2,Close.3,Close.4,Close.5,Close.6,Close.7,Close.8,...,Volume.10,Volume.11,Volume.12,Volume.13,Volume.14,Volume.15,Volume.16,Volume.17,Volume.18,Volume.19
0,Ticker,AAPL,AMZN,BA,CAT,CVX,GOOGL,GS,JNJ,JPM,...,MCD,MSFT,NKE,NVDA,PFE,SOFI,TSLA,UNH,WMT,XOM
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22/1/2024,192.9447479,154.7799988,214.9299927,284.3869019,136.6049042,145.4626923,376.8315735,157.4842072,166.308197,...,3250200,27016900,9897400,452955000,41663900,69046700,117952500,5922900,13938300,19955900
3,23/1/2024,194.2284546,156.0200043,211.5,284.5444946,136.2214203,146.5088959,371.7643433,154.905838,165.2132263,...,1874000,20525900,8208000,294654000,33466700,45600700,106605900,2672900,12066600,15863400
4,24/1/2024,193.5517731,156.8699951,214.1300049,286.3670654,138.9345398,148.1629028,370.4267578,154.0819397,166.6894531,...,3209700,24867000,7879300,560271000,42720700,41431000,123369900,2918800,22932900,17330600


In [105]:
import pandas as pd

# 1. Load the file each time from scratch
df = pd.read_csv("Trading_Project_Data.csv")

# 2. Extract ticker names from row 0
tickers = df.iloc[0, 1:21].astype(str).tolist()

# 3. Remove the first two rows, rename Date
df = df.iloc[2:].reset_index(drop=True)
df.rename(columns={'Price': 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# 4. Extract only the next 20 columns for prices
data = df.iloc[:, 1:21].copy()
data.columns = tickers
data.insert(0, 'Date', df['Date'])

# 5. Clean & convert to numeric
data.set_index('Date', inplace=True)
data = data.apply(pd.to_numeric, errors='coerce')
data.dropna(axis=1, how='all', inplace=True)

# 6. Filter your date range using boolean indexing (which avoids the KeyError)
start_date = pd.Timestamp("2024-01-17")
end_date = pd.Timestamp("2024-03-01")
filtered_data = data[(data.index >= start_date) & (data.index <= end_date)].copy()


# 7. Calculate returns, volatility, momentum
returns = filtered_data.pct_change().dropna()
volatility = returns.std()

rolling_mean = filtered_data.rolling(window=min(5, len(filtered_data))).mean()
momentum = (filtered_data.iloc[-1] / rolling_mean.iloc[-1]).dropna()

# 8. Rank
volatility_rank = volatility.rank(ascending=False, method='first')
momentum_rank = momentum.abs().rank(ascending=False, method='first')
total_rank = volatility_rank.add(momentum_rank, fill_value=0)


# 9. Select top 10
selected_stocks = total_rank.nsmallest(20).index.tolist()
print("Selected Stocks for Trading:", selected_stocks)

#Selected Stocks: ['NVDA', 'SOFI', 'AMZN', 'CAT', 'TSLA', 'GOOGL', 'MSFT', 'XOM', 'CVX', 'BA'] 
#However, we do not want to have more than 1 stock in 1 industry, and we also want to not include those in Energy. 
#This removes XOM, MSFT, and CVX, and add BA, GS, and UNH. 


Selected Stocks for Trading: ['NVDA', 'SOFI', 'AMZN', 'CAT', 'TSLA', 'XOM', 'GOOGL', 'MSFT', 'CVX', 'BA', 'GS', 'NKE', 'JPM', 'JNJ', 'MCD', 'PFE', 'UNH', 'WMT', 'AAPL', 'KO']


In [4]:


# Read CSV using two header rows; adjust skiprows if needed.
df = pd.read_csv('Trading_Project_Data.csv', header=[0, 1], skiprows=[2])

# Convert the first column to datetime and set it as the index
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
df.set_index(df.columns[0], inplace=True)

# Rename the index to 'Date'
df.index.name = 'Date'

# Create a dictionary splitting the DataFrame by ticker symbol using the second level of columns
tickers = df.columns.get_level_values(1).unique()
stock_dfs = {ticker: df.xs(ticker, axis=1, level=1) for ticker in tickers}

# For the selected stocks, export them to CSV files
selected_tickers = ['TSLA', 'SOFI', 'NVDA', 'BA', 'NKE', 'GOOGL', 'AMZN', 'UNH', 'GS', 'CAT']
for ticker in selected_tickers:
    stock_dfs[ticker].to_csv(f"{ticker}.csv")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16300\391757083.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
c:\Users\ASUS\anaconda3\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
